# Modelagem preditiva
_Machine Learning_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
3. **Preparação dos dados**
    - 3.1. Exibição dos metadados
    - 3.2. Preparação das bases de treino e teste
        - 3.2.1. Análise de cardinalidade
        - 3.2.2. Variáveis não aplicáveis à modelagem
        - 3.2.3. Segmentação das bases **train** e **test**
        - 3.2.4. Transformação das features das bases **train** e **test**
4. **Modelagem preditiva**


<br>

---

<br>

## 1. Importação de bibliotecas

In [1]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import time
import optuna

from pathlib import Path

d:\PROJETOS\deteccao_de_fraude_em_transacoes_financeiras\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar para exibir todas as colunas do Dataframe
pd.set_option('display.max_columns', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar o estilo do Seaborn
sns.set(style='whitegrid')

## 2. Carregamento das bases

In [3]:
# Efetuando a limpeza de memória antes do carregamento dos dados

print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')


Quantidade de objetos removidos da memória: 0


In [4]:
# Caminho base
base_path = Path('dados/dados_transformados_parquet/')

# Mapeamento nome_variavel: nome_arquivo
files = {
    'df_train': 'df_train.parquet',
    'df_val': 'df_val.parquet',
    'df_test': 'df_test.parquet'
}

# Carrega os DataFrames no dicionário
temp_dfs = {}
for var_name, file_name in files.items():
    try:
        temp_dfs[var_name] = pd.read_parquet(base_path / file_name)
    except Exception as e:
        print(f'Erro ao carregar {file_name}: {e}')

# Atribui às variáveis explicitamente
df_train, df_val, df_test = [temp_dfs[name] for name in ['df_train', 'df_val', 'df_test']]


In [5]:
# Exibindo da volumetria dos dados

print('\nVOLUMETRIA')
for name, df in temp_dfs.items():
    print(f'\n{name}')
    print('-' * 45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]:,}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]:,}')



VOLUMETRIA

df_train
---------------------------------------------
Quantidade de linhas (registros):  3,566,068
Quantidade de colunas (variáveis): 157

df_val
---------------------------------------------
Quantidade de linhas (registros):  891,518
Quantidade de colunas (variáveis): 157

df_test
---------------------------------------------
Quantidade de linhas (registros):  2,194,300
Quantidade de colunas (variáveis): 157


## 3. Preparação dos dados

### 3.1. Exibição dos metadados

In [6]:
# Função para geração de um dataframe de metadados

def generate_metadata(dataframe):
    '''
    Gera um DataFrame contendo metadados das colunas do DataFrame fornecido.

    :param dataframe: DataFrame
        DataFrame para o qual os metadados serão gerados.
    :return: DataFrame
        DataFrame contendo os metadados.
    '''
    metadata = pd.DataFrame({
        'Variável': dataframe.columns,
        'Tipo': dataframe.dtypes,
        'Qtde de nulos': dataframe.isnull().sum(),
        '% de nulos': round((dataframe.isnull().sum() / len(dataframe)) * 100, 2),
        'Cardinalidade': dataframe.nunique(),
    }).sort_values(by='Qtde de nulos', ascending=False).reset_index(drop=True)

    return metadata


In [7]:
# Exibe todas as linhas temporariamente
with pd.option_context('display.max_rows', None):
    display(generate_metadata(df_train))


,Variável,Tipo,Qtde de nulos,% de nulos,Cardinalidade
0,errors,object,3509183,98.40,22
1,card_std_last_1h,float64,3330323,93.39,76361
2,card_std_last_2h,float64,3257852,91.36,94894
3,client_std_last_1h,float64,3208485,89.97,103217
4,card_std_last_4h,float64,3138945,88.02,127133
5,client_std_last_2h,float64,3061631,85.85,144174
6,card_std_last_8h,float64,2952306,82.79,186123
7,client_std_last_4h,float64,2829237,79.34,222446
8,card_std_last_12h,float64,2809989,78.80,241142
9,client_std_last_8h,float64,2481589,69.59,373961


### 3.2. Preparação das bases de treino e teste

#### 3.2.1. Análise de cardinalidade

In [8]:
# Listando variáveis com determinado valor de cardinalidade

def list_low_cardinality_columns(dataframe, threshold=2):
    '''
    Lista os nomes das colunas de um DataFrame com cardinalidade inferior a um valor dado.

    :param dataframe: pd.DataFrame
        DataFrame a ser analisado.
    :param threshold: int
        Valor de corte para cardinalidade. Default é 2.
    :return: list
        Lista com os nomes das colunas cuja cardinalidade é menor que o threshold.
    '''
    return [col for col in dataframe.columns if dataframe[col].nunique(dropna=False) < threshold]


In [9]:
print('df_train - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_train))
print('\ndf_val - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_val))
print('\ndf_test - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_test))

df_train - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'flag_risky_chip_use', 'flag_no_chip_darkweb']

df_val - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'flag_risky_chip_use', 'flag_no_chip_darkweb']

df_test - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'transaction_id', 'is_fraud', 'flag_risky_chip_use', 'flag_no_chip_darkweb']


#### 3.2.2. Variáveis não aplicáveis à modelagem

In [10]:
# Lista as features de um DataFrame que contêm o termo 'id' no nome.

def list_columns_with_id(dataframe, case_sensitive=False):
    '''
    Lista os nomes das colunas de um DataFrame que contêm 'id' no nome.

    :param dataframe: pd.DataFrame
        DataFrame a ser analisado.
    :param case_sensitive: bool
        Se True, diferencia maiúsculas de minúsculas. Default é False.
    :return: list
        Lista com os nomes das colunas que contêm 'id'.
    '''
    if case_sensitive:
        return [col for col in dataframe.columns if 'id' in col]
    else:
        return [col for col in dataframe.columns if 'id' in col.lower()]


In [11]:
print('df_train - variáveis com termo "id":\n', list_columns_with_id(df_train))
print('\ndf_val - variáveis com termo "id":\n', list_columns_with_id(df_val))
print('\ndf_test - variáveis com termo "id":\n', list_columns_with_id(df_test))

df_train - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']

df_val - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']

df_test - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']


In [12]:
# Criando conjunto de variáveis que serão removidas dos DataFrames

vars_to_remove = ['zip', 'id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 
                  'id_client', 'transaction_id', 'merchant_id_is_new','card_on_dark_web', 
                  'flag_risky_chip_use', 'flag_no_chip_darkweb']

#### 3.2.3. Segmentação das bases **train** e **test**

In [13]:
# Separando as variáveis preditivas e a variável preditora (alvo)

features = df_train.columns.drop('is_fraud')
features = features.drop(vars_to_remove, errors='ignore')
target = 'is_fraud'

In [14]:
# Separando as variáveis numéricas e categóricas

numerical_features = df_train[features].select_dtypes(exclude=object).columns
categorical_features = df_train[features].select_dtypes(include=object).columns

In [15]:
# Converter todas as colunas categóricas para string

df_train[categorical_features] = df_train[categorical_features].astype(str)
df_val[categorical_features] = df_val[categorical_features].astype(str)
df_test[categorical_features] = df_test[categorical_features].astype(str)

In [16]:
# Removendo as variáveis dos DataFrames de treino e teste

X_train = df_train[features]
y_train = df_train[target]

X_test = df_val[features]
y_test = df_val[target]

#### 3.2.4. Transformação das features das bases **train** e **test**